문장을 띄어쓰기 만으로 토크나이징을 할 수도 있습니다. 데이터의 숫자가 정말로 풍부하고, 모델이 수많은 어절을 그대로 학습할 수 있을만큼의 하드웨어라면, 어절을 그대로 Word2Vec 학습하여도 그 결과는 충분히 쓸만합니다만, 그 목적이 다를 수 있습니다. 아래에서 그 예시를 살펴보겠습니다. 

In [1]:
import config
from navernews_10days import get_news_paths

path = get_news_paths(date='2016-10-20', tokenize=None)

soynlp=0.0.493
added lovit_textmining_dataset


soynlp.utils.DoublespaceLineCorpus 를 이용하여 list of str 이 yield 되는 input data 를 만듭니다.

In [2]:
from soynlp.utils import DoublespaceLineCorpus
    
class Word2VecCorpus:
    def __init__(self, fname):
        self.corpus = DoublespaceLineCorpus(fname, iter_sent=True)
    def __iter__(self):
        for sent in self.corpus:
            yield sent.split()
                
word2vec_corpus = Word2VecCorpus(path)
for num_sent, sent in enumerate(word2vec_corpus):
    if num_sent > 5:
        break
    print(sent[:10])

['19']
['1990']
['52', '1', '22']
['오패산터널', '총격전', '용의자', '검거', '서울', '연합뉴스', '경찰', '관계자들이', '19일', '오후']
['서울', '연합뉴스', '김은경', '기자', '사제', '총기로', '경찰을', '살해한', '범인', '성모']
['경찰에', '따르면', '성씨는', '19일', '오후', '강북경찰서', '인근', '부동산', '업소', '밖에서']


In [3]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(word2vec_corpus, min_count=30)

In [4]:
queries = '김무성 박근혜 문재인 국방부 정부 국정원 대통령 축구 외교 정책 군대 미국 일본 중국 아이오아이'.split()

어절만 이용하여 학습했음에도 불구하고, 김무성의 유사어로 정치인들이 등장함을 볼 수 있습니다. 이는 뉴스 코퍼스에서는 정치인의 이름만 적고 띄어쓰기를 하는 경우도 충분하기 때문입니다. 

하지만 '김무성 - 대표도'와 같은 유사 어절이 등장한 것은, 뉴스에서 '김무성'으로만 적는 경우와 '김무성 대표도'로 적는 경우가 혼재되어 있어서, '대표도'의 문맥이 '김무성'의 문맥과 유사하였기 때문입니다. 

비슷한 의미로, 박근혜 - 박 (6251)의 경우에는 '박근혜 대통령', '박 대통령'이 번갈아가며 이용되었기 때문입니다. 


그러나, 국정원의 경우에는 총 42번 나왔으며, min_count=30으로 하였기 때문에 18000여개의 단어 중에서는 infrequent 한 편에 속합니다. 그리고 이 때의 유사 어절들은 문맥상 잘 어울리지 않는 어절들입니다. 

    Vocab(count:42, index:12945, sample_int:4294967296)
    국정원 - 단국대 (33) (0.774)
    국정원 - 대우학원 (34) (0.726)
    국정원 - 김형수 (92) (0.722)
    국정원 - 인터뷰가 (36) (0.717)
    국정원 - 입 (59) (0.711)
    국정원 - 정동구 (31) (0.709)
    국정원 - 정유라씨가 (41) (0.705)
    국정원 - 취재진이 (33) (0.703)
    국정원 - 크리스 (33) (0.696)
    국정원 - 이사장을 (81) (0.696)
    
Word2Vec에서는 infrequent 한 단어(어절)들의 유사 단어(어절)들은 infrequent 한 경향이 있습니다. 빈도수가 충분하지 않을 경우에는 학습이 잘 되지 않는 것이라고 해석할 수 있습니다. 

어절을 그대로 학습할 경우에는, 아래와 같이 대통령의 유사어절로 '대통령 + 조사'의 어절들이 자주 등장합니다. 
    
    Seed word = 대통령
    Vocab(count:3411, index:89, sample_int:4294967296)
    대통령 - 대통령의 (3074) (0.809)
    대통령 - 대통령을 (287) (0.784)
    대통령 - 대통령과 (437) (0.784)
    대통령 - 대통령에 (255) (0.723)
    대통령 - 대통령은 (2852) (0.716)
    대통령 - 대통령에게 (203) (0.708)
    대통령 - 대통령도 (125) (0.704)
    대통령 - 대통령이다 (54) (0.698)
    대통령 - 대통령이 (5054) (0.698)
    대통령 - 정부에서 (76) (0.687)
    
우리가 원하는 것이 Language model을 학습하는 것이라면, 이 결과는 유용합니다. 하지만, 우리가 문맥적으로 유사한 단어들을 찾기 위하여 Word2Vec을 학습하는 것이 목적이었다면, 어절을 그대로 학습하는 것보다 토크나이징을 한 뒤, 이를 이용하여 학습해야 합니다. 

In [5]:
for query in queries:

    if not (query in word2vec_model.wv.vocab):
        continue

    print('\n\nQuery = {}\n{}'.format(query, word2vec_model.wv.vocab[query]))

    for sim_word, sim in word2vec_model.most_similar(query):
        sim_count = word2vec_model.wv.vocab[sim_word].count
        print('%s - %s (%d, %.3f)' % (query, sim_word, sim_count, sim))



Query = 김무성
Vocab(count:40, index:13718, sample_int:4294967296)
김무성 - 김종인 (87, 0.892)
김무성 - 대표와 (232, 0.879)
김무성 - 이재오 (32, 0.867)
김무성 - 사무총장 (104, 0.856)
김무성 - 안철수 (193, 0.856)
김무성 - 오세훈 (34, 0.841)
김무성 - 정의화 (33, 0.829)
김무성 - 서울시장 (37, 0.829)
김무성 - 김문수 (69, 0.823)
김무성 - 김부겸 (39, 0.818)


Query = 박근혜
Vocab(count:1412, index:261, sample_int:4294967296)
박근혜 - 올랑드 (53, 0.786)
박근혜 - 박 (2276, 0.785)
박근혜 - 두테르테 (334, 0.757)
박근혜 - 노 (190, 0.742)
박근혜 - 푸틴 (124, 0.732)
박근혜 - 오바마 (229, 0.713)
박근혜 - 정권 (183, 0.677)
박근혜 - 노무현 (266, 0.665)
박근혜 - 대통령이 (1454, 0.648)
박근혜 - 블라디미르 (73, 0.646)


Query = 문재인
Vocab(count:933, index:411, sample_int:4294967296)
문재인 - 안철수 (193, 0.878)
문재인 - 문 (742, 0.871)
문재인 - 반기문 (136, 0.842)
문재인 - 송민순 (814, 0.793)
문재인 - 당내 (73, 0.788)
문재인 - 김종인 (87, 0.784)
문재인 - 송 (339, 0.781)
문재인 - 대표의 (613, 0.780)
문재인 - 탈당 (189, 0.779)
문재인 - 색깔론 (68, 0.779)


Query = 국방부
Vocab(count:327, index:1515, sample_int:4294967296)
국방부 - 한민구 (143, 0.906)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):



국방부 - 카터 (207, 0.901)
국방부 - 국방장관이 (83, 0.860)
국방부 - 케리 (401, 0.859)
국방부 - 애슈턴 (130, 0.859)
국방부 - 윤병세 (297, 0.858)
국방부 - 산업통상자원부 (114, 0.846)
국방부 - 외교부 (418, 0.839)
국방부 - 참석하는 (94, 0.833)
국방부 - 국무장관과 (40, 0.832)


Query = 정부
Vocab(count:1005, index:380, sample_int:4294967296)
정부 - 정부의 (677, 0.888)
정부 - 정부와 (203, 0.822)
정부 - 예산 (305, 0.778)
정부 - 현 (689, 0.762)
정부 - 정부는 (726, 0.760)
정부 - 정부에서 (98, 0.757)
정부 - 보수 (88, 0.756)
정부 - 누리과정 (45, 0.744)
정부 - 내년도 (114, 0.732)
정부 - 구조조정 (166, 0.729)


Query = 국정원
Vocab(count:284, index:1794, sample_int:4294967296)
국정원 - 국감에서 (136, 0.812)
국정원 - 이병호 (366, 0.790)
국정원 - 정보위 (292, 0.785)
국정원 - 통화에서 (130, 0.775)
국정원 - 국가정보원 (41, 0.773)
국정원 - 연합뉴스와의 (37, 0.761)
국정원 - 김도읍 (92, 0.758)
국정원 - 국정원장의 (162, 0.747)
국정원 - 정보위원회 (121, 0.744)
국정원 - 정세균 (44, 0.742)


Query = 대통령
Vocab(count:1187, index:309, sample_int:4294967296)
대통령 - 대통령과 (197, 0.857)
대통령 - 대통령의 (856, 0.850)
대통령 - 대통령도 (53, 0.829)
대통령 - 이명박 (70, 0.824)
대통령 - 박정희 (48, 0.822)
대통령 - 김대중 (74, 0.809)
대